In [ ]:
##in this version i will say screw pipelines and just do it without for now ... 
#maybe I will see what this infinity issue is if I can inspect the resuly of each step manually

In [ ]:
##Round 1 cleaning complete. v1 decisions: keep all nan as new feature/value; no skew or outlier action has been taken
##Round 1 exploration complete. (a) Distribution of each feature explored (b) feature-target and feature-feature relationships explored 

##Round 1 modelling Plan: make one model for each notebook, try to do systematic tuning ... take best few models (?) or is it too soon?
##then go back and do feature engineering and feature selection based on these base-line models
##Build pipe-line: dummies, scaling, tuning ...

##next steps:
##
##Flask API Deployment
##build small webpage interaction


### Tuning of KNN 

Main parameters: k = num_neighbors, and metric

In [ ]:
## following https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

In [ ]:
##brief review of KNN vs K-means ... 
##K-means (unspervised) has no labels, just feature space ... you create k-labels (choice of k is tricky and important)
##then run thru k-means clustering algorithm to assign k-labels (clusters) to the data 
##b
##KNN (supervised) is simpler because data already comes with p-labels (the role of k is different here) 
##in the training data on the test data you simply look at the k-nearet neighors (of training set) and take majority vote 
##on what label to assign to new point. The choice of k is important and the metric can change the outcomes.

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score
import seaborn as sns

import pickle

#sns.set()
sns.set(rc={'figure.figsize':(8,5)})



In [2]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [3]:
path = "/Users/joejohns/Downloads/data_loans.csv"

data = pd.read_csv(path) 
print(data.shape)
data.head(3)

(614, 13)


Loan_ID Gender Married Dependents Education Self_Employed  \
0  LP001002   Male      No          0  Graduate            No   
1  LP001003   Male     Yes          1  Graduate            No   
2  LP001005   Male     Yes          0  Graduate           Yes   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0         NaN             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   

   Credit_History Property_Area Loan_Status  
0             1.0         Urban           Y  
1             1.0         Rural           N  
2             1.0         Urban           Y

## data processing from round 1


In [4]:
perc_null(data).index

    

Index(['Credit_History', 'Self_Employed', 'LoanAmount', 'Dependents',
       'Loan_Amount_Term', 'Gender', 'Married', 'Loan_ID', 'Education',
       'ApplicantIncome', 'CoapplicantIncome', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [5]:
data.rename(columns = {'ApplicantIncome':'Applicant_Income', 'CoapplicantIncome': 'Coapplicant_Income', 'LoanAmount':'Loan_Amount'}, inplace = True)

In [6]:
target = ['Loan_Status']
ID_feats = ['Loan_ID',] 

#feature types:

numer_feats = ['Applicant_Income', 'Coapplicant_Income', 'Loan_Amount', 'Loan_Amount_Term',]

ord_cat_feats = [ 'Dependents', 'Education', ]
nom_cat_feats = ['Gender', 'Married', 'Self_Employed', 'Credit_History', 'Property_Area',]



In [7]:
##fix ordinal cats (write general function which does this, no ordinal encoder)
for item in ['0', '1', '2', '3+']:
    if item != '3+':
        i = int(item)
    else:
        i = 3
    data.loc[data['Dependents'] == item,'Dependents']  = i


In [8]:
#fix target; make it numeric; useful for regression modelling

data.loc[data["Loan_Status"] == 'Y', "Loan_Status"] = 1
data.loc[data["Loan_Status"] == 'N', "Loan_Status"] = 0



In [9]:
#fix this nom cat ... float --> string

data.loc[data['Credit_History'] ==0.0, 'Credit_History'] = 'No'
data.loc[data['Credit_History'] == 1.0, 'Credit_History'] = 'Yes'  #I use capitalized "Yes", "No" to be conistent with their other entries

In [10]:
#turn this into ordinal cat

data.loc[data['Education']=='Not Graduate', "Education"] = 0
data.loc[data['Education']=='Graduate', "Education"] = 1

In [11]:
#set up some feature lists for imputing and scaling 

#steps:

#1. create new cat recording if NaN in specified numeric or nominal features 
#(do this before imputing of course)

#numeric and nominal: 
#2. impute most_freq (Loan_Term)
#3. impute mean (everyone else)
#4. scale by max, min (or by mean; try both)

#nominal cats:
#5. get_dummies, nan yes
#6. get_dummies, nan no

impute_most_freq = ['Loan_Amount_Term']
impute_mean = [x for x in ord_cat_feats+numer_feats if x not in impute_most_freq] 
 
scale = ord_cat_feats+numer_feats

#can make a for loop that extracts these ... 
numer_ord_nan_new_cat = [ 'Loan_Amount', 'Loan_Amount_Term', 'Dependents', ] #make new category for these 'unk' = yes/no; ##impute by mean
nom_cat_nan_dummies = ['Credit_History', 'Self_Employed', 'Gender', 'Married',]  ##make new dumies cat_val for nan
nom_cat_dummies = ['Property_Area',]  #regular dummies



In [12]:
##carry out imputation and scaling steps

In [15]:
#step 1. create new categorical (records if any null values in any of specified cats)

data['missing_data'] = 'No'  
for feature in numer_ord_nan_new_cat:
    data.loc[(data[feature].isnull()) , 'missing_data'] = 'Yes'

    


In [16]:
#update nom_cat_feats (no null values, ok)

nom_cat_feats+= ['missing_data']

In [40]:
#general functions ... need testing; build simple df for various tests

def most_freq_imputer(df, col_list): #data =df, list_col = [col1, col2, ...] columns of df
    for col in col_list:
        most_freq_val = df[col].value_counts().index[0]  #0 grabs most frequent
        df.loc[df[col].isnull(), col] = most_freq_val


def mean_imputer(df, col_list): #data =df, list_col = [col1, col2, ...] columns of df
    for col in col_list:
        mean = round(np.mean(df[col]),2)
        df.loc[df[col].isnull(), col] = mean



In [39]:
#max_min scaler

def max_min_helper(x, min, max, new_min =0, new_max=1):
    y = (x-min)/(max-min)   #scales to min = 0, max = 1
    scaled = y*(new_max-new_min) + new_min 
    return scaled
#scale by max 

def max_min_scaler(df, col_list, new_min =0, new_max=1):
    for col in col_list:

        min = np.min(df[col])
        max = np.max(df[col])
        
        df[col] = df[col].map(lambda x: max_min_helper(x, min, max, new_min, new_max))
                   
    
##why is this one not working? because you has max, min arguments switched in your function call

In [41]:
#mean scaler


def mean_scaler(df, col_list):
    for col in col_list:
        mean = np.mean(df[col])
        sigma = np.std(df[col]) #std deviation
        df[col] = (df[col] - mean)/sigma
              

In [37]:
##tests of these functions

data_dic = {'Name':['Tom', 'Jack', 'Steve', 'Ricky',np.NaN, 'Jack'],'Age':[28,34,29,42, 29, np.NaN], "Salary":[50,50,50,np.NaN ,60,30]}
df_testing = pd.DataFrame(data_dic)
df_testing

Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0     NaN
4    NaN  29.0    60.0
5   Jack   NaN    30.0

In [45]:
for func in [most_freq_imputer ]:
    col_list = ['Name']
    df = df_fn_tests.copy()
    print(func)
    print(df)
    func(df,col_list )
    print(" ")
    print(df)
    print("**** ")

<function most_freq_imputer at 0x7fab2a3eaca0>
    Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0     NaN
4    NaN  29.0    60.0
5   Jack   NaN    30.0
 
    Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0     NaN
4   Jack  29.0    60.0
5   Jack   NaN    30.0
**** 


In [44]:
##tests

#fortunately, all functions have same input function(df, col_list)
for func in [most_freq_imputer, mean_imputer,  max_min_scaler, mean_scaler  ]:
    col_list = ['Salary','Age']
    df = df_fn_tests.copy()
    print(func)
    print(df)
    func(df,col_list )
    print(" ")
    print(df)
    print("**** ")

<function most_freq_imputer at 0x7fab2a3eaca0>
    Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0     NaN
4    NaN  29.0    60.0
5   Jack   NaN    30.0
 
    Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0    50.0
4    NaN  29.0    60.0
5   Jack  29.0    30.0
**** 
<function mean_imputer at 0x7fab2a3eae50>
    Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0     NaN
4    NaN  29.0    60.0
5   Jack   NaN    30.0
 
    Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0    48.0
4    NaN  29.0    60.0
5   Jack  32.4    30.0
**** 
<function max_min_scaler at 0x7fab2a3eaaf0>
    Name   Age  Salary
0    Tom  28.0    50.0
1   Jack  34.0    50.0
2  Steve  29.0    50.0
3  Ricky  42.0     NaN
4    NaN  29.0    60.0
5   Jack   NaN    30.0
 
    Name       Age    Salary
0    Tom  0.000

In [27]:
##impute most frequent 

most_freq_val = data['Loan_Amount_Term'].value_counts().index[0].copy()  #0 grabs most frequent

data.loc[data['Loan_Amount_Term'].isnull(), 'Loan_Amount_Term'] = most_freq_val

#for feat in impute_most_freq:
    
    

In [38]:
#check that col has no null
#perc_null(data)

In [36]:
##impute mean # start on one column

mean = round(np.mean(data['Loan_Amount']),2)

data.loc[data['Loan_Amount'].isnull(), 'Loan_Amount'] = mean

In [56]:
#test on one column


def max_min_scale(x, max, min, new_min =0, new_max=1):
    y = (x-min)/(max-min)   #scales to min = 0, max = 1
    scaled = y*(new_max-new_min) + new_min 
    return scaled

df = data.copy()
col = 'Loan_Amount'

max = np.max(df[col])
min = np.min(df[col])

df[col].map(lambda x: max_min_scale(x, max, min, new_min =0, new_max =1))

0      0.198857
1      0.172214
2      0.082489
3      0.160637
4      0.191027
         ...   
609    0.089725
610    0.044863
611    0.353111
612    0.257598
613    0.179450
Name: Loan_Amount, Length: 614, dtype: float64

In [50]:
#test mean scaler on single column 

df = data.copy()
col = 'Loan_Amount'

mean = np.mean(df[col])
sigma = np.std(df[col]) #std deviation
(df[col] - mean)/sigma

0     -0.000025
1     -0.219272
2     -0.957640
3     -0.314546
4     -0.064453
         ...   
609   -0.898094
610   -1.267278
611    1.269372
612    0.483368
613   -0.159727
Name: Loan_Amount, Length: 614, dtype: float64